In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import s3fs
import datetime as dt
import zipfile
import logging
import os
import fsspec
import ujson
import dask
from tqdm import tqdm
from glob import glob
from dask_jobqueue import SLURMCluster
from kerchunk.hdf import SingleHdf5ToZarr 
from kerchunk.combine import MultiZarrToZarr
from dask.distributed import Client

/data/keeling/a/alfonso8/miniconda3/envs/kerchunk-tutorial/lib/python3.8/site-packages/dask_jobqueue/core.py:20: FutureWarning: tmpfile is deprecated and will be removed in a future release. Please use dask.utils.tmpfile instead.
  from distributed.utils import tmpfile


In [2]:
cluster = SLURMCluster(
                       queue="seseml",
                       memory='10GB',
                       cores=10,
                       processes=1,
                       walltime='02:30:00',
                       scheduler_options={
                                          'host': '172.22.179.3:7333', 
                                          'dashboard_address': '7999',
                                          },                        
                      )

In [3]:
cluster.scale(8)

In [4]:
cluster

Dashboard: /proxy/7999/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://172.22.179.3:7333,Workers: 0
Dashboard: /proxy/7999/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [5]:
client = Client(cluster)

In [6]:
client

Connection method: Cluster object,Cluster type: dask_jobqueue.SLURMCluster
Dashboard: /proxy/7999/status,
Dashboard: /proxy/7999/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://172.22.179.3:7333,Workers: 0
Dashboard: /proxy/7999/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [7]:
fs = fsspec.filesystem('s3', anon=True)

urls = ['s3://' + f for f in fs.glob("s3://noaa-goes16/ABI-L2-SSTF/2020/210/*/*.nc")]

In [8]:
def gen_json(u):
    so = dict(
        mode="rb", anon=True, default_fill_cache=False, default_cache_type="none"
    )
    with fsspec.open(u, **so) as inf:
        h5chunks = SingleHdf5ToZarr(inf, u, inline_threshold=300)
        with open(f"jsons/{u.split('/')[-1]}.json", 'wb') as outf:
            outf.write(ujson.dumps(h5chunks.translate()).encode())

In [9]:
# Create json/ folder if it doesn't already exist
import pathlib
pathlib.Path('./jsons/').mkdir(exist_ok=True)

In [10]:
_ = dask.compute(*[dask.delayed(gen_json)(u) for u in urls])

In [11]:
json_list = sorted(glob("./jsons/*.json"))

In [12]:
m_list = []
for j in tqdm(json_list):
    with open(j) as f:
        m_list.append(fsspec.get_mapper("reference://", 
                        fo=ujson.load(f),
                        remote_protocol='s3',
                        remote_options={'anon':True}))

100%|███████████████████████████████████████████████████████████████████████████████████| 24/24 [00:00<00:00, 52.73it/s]


In [13]:
%%time
ds = xr.open_mfdataset(m_list, engine='zarr', combine='nested', concat_dim='t', 
                        coords='minimal', data_vars='minimal', compat='override',
                        parallel=True)
ds

CPU times: user 1.97 s, sys: 222 ms, total: 2.19 s
Wall time: 5.01 s


<xarray.Dataset>
Dimensions:                                                 (t: 24, y: 5424,
                                                             x: 5424,
                                                             SST_day_night_emissive_bands: 4,
                                                             SST_night_only_emissive_band: 1,
                                                             number_of_SZA_bounds: 2,
                                                             number_of_LZA_bounds: 2,
                                                             number_of_time_bounds: 2,
                                                             number_of_image_bounds: 2)
Coordinates: (12/14)
    SST_day_night_emissive_band_ids                         (SST_day_night_emissive_bands) float32 dask.array<chunksize=(4,), meta=np.ndarray>
    SST_day_night_emissive_wavelengths                      (SST_day_night_emissive_bands) float32 dask.array<chunksize=(4,), meta=np.ndarray>
    SST_night_only_emissive_band_id                         (SST_night_only_emissive_band) float32 dask.array<chunksize=(1,), meta=np.ndarray>
    SST_night_only_emissive_wavelength                      (SST_night_only_emissive_band) float32 dask.array<chunksize=(1,), meta=np.ndarray>
    day_solar_zenith_angle                                  float32 ...
    night_solar_zenith_angle                                float32 ...
    ...                                                      ...
    retrieval_solar_zenith_angle                            float32 ...
  * t                                                       (t) datetime64[ns] ...
  * x                                                       (x) float32 -0.15...
    x_image                                                 float32 ...
  * y                                                       (y) float32 0.151...
    y_image                                                 float32 ...
Dimensions without coordinates: SST_day_night_emissive_bands,
                                SST_night_only_emissive_band,
                                number_of_SZA_bounds, number_of_LZA_bounds,
                                number_of_time_bounds, number_of_image_bounds
Data variables: (12/42)
    DQF                                                     (t, y, x) float32 dask.array<chunksize=(1, 226, 226), meta=np.ndarray>
    SST                                                     (t, y, x) float32 dask.array<chunksize=(1, 226, 226), meta=np.ndarray>
    algorithm_dynamic_input_data_container                  (t) float64 nan ....
    algorithm_product_version_container                     (t) float64 nan ....
    day_solar_zenith_angle_bounds                           (t, number_of_SZA_bounds) float32 dask.array<chunksize=(1, 2), meta=np.ndarray>
    geospatial_lat_lon_extent                               (t) float64 nan ....
    ...                                                      ...
    total_number_of_severely_degraded_quality_ocean_pixels  (t) float64 1.237...
    total_number_of_unprocessed_pixels                      (t) float64 1.256...
    twilight_solar_zenith_angle                             (t) float64 85.0 ...
    twilight_solar_zenith_angle_bounds                      (t, number_of_SZA_bounds) float32 dask.array<chunksize=(1, 2), meta=np.ndarray>
    x_image_bounds                                          (t, number_of_image_bounds) float32 dask.array<chunksize=(1, 2), meta=np.ndarray>
    y_image_bounds                                          (t, number_of_image_bounds) float32 dask.array<chunksize=(1, 2), meta=np.ndarray>
Attributes: (12/34)
    Conventions:               CF-1.7
    Metadata_Conventions:      Unidata Dataset Discovery v1.0
    _FillValue:                -999.0
    cdm_data_type:             Image
    cell_methods:              quantitative_local_zenith_angle: sum retrieval...
    dataset_name:              OR_ABI-L2-SSTF-M6_G16_s20202100000205_e2020210...
  

In [14]:
mzz = MultiZarrToZarr(
    json_list,
    remote_protocol="s3",
    remote_options={'anon':True},
    xarray_open_kwargs={
        "decode_cf" : False,
        "mask_and_scale" : False,
        "decode_times" : False,
        "decode_timedelta" : False,
        "use_cftime" : False,
        "decode_coords" : False
    },
    xarray_concat_args={
        'data_vars' : 'minimal',
        'coords' : 'minimal',
        'compat' : 'override',
        'join' : 'override', 
        'combine_attrs' : 'override',
        'dim' : 't'
    }
)

TypeError: __init__() got an unexpected keyword argument 'xarray_open_kwargs'